In [19]:
import scipy         
from scipy.linalg import expm
import scipy.sparse as sparse
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis, spin_basis_1d # bases
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [20]:
L = 5 #system size
M = 0.1 #fermion mass
N=4 #number of layers
l=1 #spin length
m_epsilon = 0
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

In [21]:
#construct basis
basis=spin_basis_1d(L=L,S = str(l))
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
       0.         |2 2 2 2 2>           242  
       1.         |2 2 2 2 1>           241  
       2.         |2 2 2 2 0>           240  
       3.         |2 2 2 1 2>           239  
       4.         |2 2 2 1 1>           238  
       5.         |2 2 2 1 0>           237  
       6.         |2 2 2 0 2>           236  
       7.         |2 2 2 0 1>           235  
       8.         |2 2 2 0 0>           234  
       9.         |2 2 1 2 2>           233  
      10.         |2 2 1 2 1>           232  
      11.         |2 2 1 2 0>           231  
      12.         |2 2 1 1 2>           230  
      13.         |2 2 1 1 1>           229  
      14.         |2 2 1 1 0>           228  
      15.         |2 2 1 0 2>           227  
      16.         |2 2 1 0 1>           226  
      17.         |2 2 1 0 0>           225  
      18.         |2 2 0 2 2>           224  
      19.         |2 2 0 2 1>           223  
      20.  

In [22]:
#initialisation of the Gauss laws

liste = []
liste_m = []
gauss_law_map = []
Gauss_law = []
gauss_law_matrix = []
z_comp_minf = [[-m_epsilon,0]]
z_comp_pinf = [[m_epsilon,L-1]]

for i in range(L+1):
    liste.append([[1.,i]])
    liste_m.append([[-1.,i]])
    if i == 0:
        gauss_law_map.append([
            ["z",liste[0]],
            ["I",z_comp_minf],

        ])
    
    elif (i == L) & (i%2 == 0):
        gauss_law_map.append([
            ["z",liste_m[i-1]],
            ["I",z_comp_pinf],
        ])
    
    elif (i == L) & (i%2 == 1):
        gauss_law_map.append([
            ["z",liste_m[i-1]],
            ["I",liste[i-1]],
            ["I",z_comp_pinf],

        ])        
        
    elif i%2 == 0:
        gauss_law_map.append([
            ["z",liste[i]],
            ["z",liste_m[i-1]],
        ])
    elif i%2 == 1:
        gauss_law_map.append([
            ["z",liste[i]],
            ["z",liste_m[i-1]],  
            ["I",liste[i]]
        ])
    Gauss_law.append(hamiltonian(gauss_law_map[i],dynamic_list=[],basis=basis,**no_checks))
for i in range(L+1):
    gauss_law_matrix.append(Gauss_law[i].tocsc())

In [23]:
#projectors on g=1 for every site
Proj = [sparse.csr_matrix((basis.Ns,basis.Ns)) for i in range(L+1)]
for i in range(L+1):
    for j in range(basis.Ns):
        if (np.real(gauss_law_matrix[i][j,j])>=1.-1e-8) & (np.real(gauss_law_matrix[i][j,j])<=1.+1e-8):
            Proj[i][j,j] = 1.

In [24]:
A = np.zeros((2*l+1,2*l+1))
for i in range(2*l+1):
    for j in range(2*l+1):
        A[i][j] = (-l+i)**j
b = np.zeros(2*l+1)
for i in range(2*l+1):
    b[i] = (-1)**(-l+i)
solution = scipy.linalg.solve(A,b)
x = np.zeros(2*3+1)
for i in range(2*l+1):
    x[i] = solution[i]
print(x)

[ 1.  0. -2.  0.  0.  0.  0.]


In [25]:
const_term = []
linear_term = []
quadratic_term = []
cubic_term = []
quartic_term = []
quintic_term = []
sextic_term = []
interaction_p = []
interaction_m = []

for i in range(L-1):
    const_term.append([[x[0]/(2*np.sqrt(l*(l+1))),i]])
    linear_term.append([[x[1]/(2*np.sqrt(l*(l+1))),i,i+1]])
    quadratic_term.append([[x[2]/(2*np.sqrt(l*(l+1))),i,i+1,i+1]])
    cubic_term.append([[((-1)**i*x[3])/(2*np.sqrt(l*(l+1))),i,i+1,i+1,i+1]])
    quartic_term.append([[((-1)**i*x[4])/(2*np.sqrt(l*(l+1))),i,i+1,i+1,i+1,i+1]])
    quintic_term.append([[((-1)**i*x[5])/(2*np.sqrt(l*(l+1))),i,i+1,i+1,i+1,i+1,i+1]])
    sextic_term.append([[((-1)**i*x[6])/(2*np.sqrt(l*(l+1))),i,i+1,i+1,i+1,i+1,i+1,i+1]])

    const_term

    interaction_p.append([
        ["+", const_term[i]],
        ["+z", linear_term[i]],
        ["+zz", quadratic_term[i]],
        ["+zzz", cubic_term[i]],
        ["+zzzz", quartic_term[i]],
        ["+zzzzz", quintic_term[i]],
        ["+zzzzzz", sextic_term[i]],

    ])

    interaction_m.append([
        ["-", const_term[i]],
        ["-z", linear_term[i]],
        ["-zz", quadratic_term[i]],
        ["-zzz", cubic_term[i]],
        ["-zzzz", quartic_term[i]],
        ["-zzzzz", quintic_term[i]],
        ["-zzzzzz", sextic_term[i]],

    ])

const_term.append([[(0.5*(-1)**m_epsilon)/np.sqrt(l*(l+1)),L-1]])

interaction_p.append([
    ["+", const_term[L-1]]
])
interaction_m.append([
    ["-", const_term[L-1]]
])

kin_energy = [[0.5,i,i] for i in range(L)]
mass_term = [[2*(-1)**i*M,i] for i in range(L)]

kin_mass_map = [
            ["zz", kin_energy], 
            ["z", mass_term], 
]

H_int_p = []
H_int_m = []
interaction = [sparse.csr_matrix((basis.Ns,basis.Ns)) for i in range(L)]
h_int = sparse.csr_matrix((basis.Ns,basis.Ns))
for i in range(L):
    H_int_p.append(hamiltonian(interaction_p[i],dynamic_list=[],basis=basis,**no_checks))
    H_int_m.append(hamiltonian(interaction_m[i],dynamic_list=[],basis=basis,**no_checks))
    interaction[i] = (Proj[i]@H_int_p[i].tocsc()@Proj[i+1]+Proj[i+1]@H_int_m[i].tocsc()@Proj[i])

H_kin_mass = hamiltonian(kin_mass_map,dynamic_list=[],basis=basis,**no_checks)
h_kin_mass = H_kin_mass.tocsc()

h_int = 0

for i in range(L):
    h_int += interaction[i]


full_ham_matrix = h_kin_mass + h_int #Spin-1 Hamiltonian without the local contstaints

G=0
for i in range(L+1):
    G += 0.12*(Gauss_law[i]**2-Gauss_law[i])**2
constraint_full_matrix = G.tocsc()
constrained_full_ham_matrix= full_ham_matrix + constraint_full_matrix #Spin-1 Hamiltonian with the local contstaints as a penalty term
#print(full_ham_matrix)

In [26]:
#eigenvalues and eigenvectors of the Spin-1 Hamiltonian without the local contstaints
eigenval, eigenvec = scipy.sparse.linalg.eigsh(full_ham_matrix,k = 3,which = "SA")
print(eigenval[eigenval<=1e1])
print(eigenvec[:,0])

[-1.0416023  -0.34551385 -0.34551385]
[-3.73561286e-03+1.72762984e-03j -1.15785589e-17+4.52221293e-18j
 -2.64277565e-18+3.20672296e-18j -1.39771777e-02+6.46410379e-03j
  2.61283389e-02-1.20837195e-02j  2.64470046e-17+1.49795576e-17j
 -1.24431657e-17+8.04120210e-17j  2.10513722e-18+2.36475411e-18j
 -2.57988098e-18-1.34367490e-17j  1.03193217e-17+1.72655502e-20j
 -1.45409369e-17-6.60255548e-18j -2.89329693e-17-1.00263621e-17j
 -4.46146822e-18+5.20655006e-18j  1.30567846e-17+3.58014287e-18j
  8.63252707e-18-4.96774470e-18j  2.98476929e-18+3.10802479e-17j
 -1.85626011e-17-1.20032891e-17j  9.05080801e-18+2.31397588e-18j
 -1.51040077e-18-2.74090053e-17j -3.57407530e-17+5.56880580e-18j
 -4.60747418e-18+1.60857061e-18j  1.72019168e-17-3.46231017e-18j
  1.62477521e-17-2.17774296e-18j -8.76432363e-19+2.44748157e-18j
  2.61969412e-18+5.08113577e-18j  4.32408040e-18-3.12208712e-18j
 -1.99006129e-17+1.22429686e-17j -1.39771777e-02+6.46410379e-03j
  3.92784082e-18+3.65967651e-17j -4.68561792e-18+2.6

In [27]:
#eigenvalues and eigenvectors of the Spin-1 Hamiltonian with the local contstaints
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(constrained_full_ham_matrix,k = 3,which = "SA")


In [28]:
#Gauge invariant part of the spectrum
string = ""
print(eigenvalues[eigenvalues<=1e1])
for i in range(basis.Ns):
    if np.abs(eigenvectors[:,0][i])>=1e-5:
        string+=str(eigenvectors[:,0][i])+basis.int_to_state(basis.Ns-i-1)+"\n"
        #for j in range(basis.Ns):
            #if (np.abs(np.abs(eigenvectors[:,0][i])-np.abs(eigenvectors[:,0][j]))<=1e-5) & (i<j):
                #print(basis.int_to_state(basis.Ns-i-1),basis.int_to_state(basis.Ns-j-1))
print(string)


[-1.0416023  -0.0142565   0.15174042]
(-0.0011939674467996936-0.00393877519793633j)|2 2 2 2 2>
(-0.0044673513491718785-0.014737330353310134j)|2 2 2 1 2>
(0.008351075776187803+0.02754933581439016j)|2 2 2 1 1>
(-0.004467351349171878-0.014737330353310173j)|2 1 2 2 2>
(-0.021204650164136935-0.06995194916853513j)|2 1 2 1 2>
(0.041323305819137096+0.13632131470034786j)|2 1 2 1 1>
(0.041651841261742956+0.13740511916792172j)|2 1 1 1 2>
(-0.09109490648490119-0.30051268087985344j)|2 1 1 1 1>
(0.034348095480921714+0.11331081675570742j)|2 1 1 0 1>
(0.008351075776187765+0.027549335814390234j)|1 1 2 2 2>
(0.04132330581913694+0.13632131470034786j)|1 1 2 1 2>
(-0.08113952492185998-0.2676709060964609j)|1 1 2 1 1>
(-0.09109490648490082-0.3005126808798534j)|1 1 1 1 2>
(0.199978955368748+0.6597099038394192j)|1 1 1 1 1>
(-0.07663387160472772-0.252807221509758j)|1 1 1 0 1>
(0.03434809548092157+0.11331081675570741j)|1 0 1 1 2>
(-0.07663387160472754-0.252807221509758j)|1 0 1 1 1>
(0.03260440376615935+0.1075585

In [29]:
#initial state
string = ""
for i in range(L):
    string +="1"
psi_0 = np.zeros(basis.Ns)
i_0 = basis.index(string)
psi_0[i_0] = 1.

In [30]:
#cost function of the initial state
expt_value = 0
matvec_h = full_ham_matrix@psi_0
matvec_c = constraint_full_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*(matvec_h[i]+matvec_c[i])
print(np.real(expt_value))

0.0


In [31]:
expt_value = 0
matvec = full_ham_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*matvec[i]
print(expt_value)

0j


In [45]:
def rot_sigma(k,i,j,phi,theta):
    if k>=L:
        print("Warning! Rotations apply to k<=L-1!")
    sigma = np.zeros((2*l+1,2*l+1),dtype = complex)
    sigma[2*l-i][2*l-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2*l-j][2*l-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    sigma = sparse.csc_matrix(sigma)
    rot_matrix = expm(-1j*theta*sigma/2)
    if k==0:
        tensprod = rot_matrix
        for i in range(L-1):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
    else:
        tensprod = sparse.identity(2*l+1)
        for i in range(1,k):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        tensprod = scipy.sparse.kron(tensprod,rot_matrix)
        for i in range(k+1,L):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
    return tensprod

def rot_ms(k,i,j,phi,theta):
    sigma = np.zeros((2*l+1,2*l+1),dtype = complex)
    sigma[2*l-i][2*l-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2*l-j][2*l-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    sigma = sparse.csc_matrix(sigma)
    power_matrix = scipy.sparse.kron(sigma,sparse.identity(2*l+1))+scipy.sparse.kron(sparse.identity(2*l+1),sigma)
    rot_matrix = scipy.sparse.linalg.expm(-1j*power_matrix@power_matrix*theta/4)
    if k == 0:
        tensprod = rot_matrix
        for i in range(L-2):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
    else:
        tensprod = sparse.identity(2*l+1)
        for i in range(1,k):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        tensprod = scipy.sparse.kron(tensprod,rot_matrix)
        for i in range(k+2,L):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
    return tensprod

def rot_ms_e(i,j,phi,theta):
    ms_e = sparse.identity(basis.Ns,dtype = complex)
    for k in range(1,int(L/2)):
        ms_e = rot_ms(2*k,i,j,phi,theta)@ms_e
    return(ms_e)

def rot_ms_o(i,j,phi,theta):
    ms_o = sparse.identity(basis.Ns,dtype = complex)
    for k in range(int(L/2)-1):
        ms_o = rot_ms(2*k+1,i,j,phi,theta)@ms_o
    return(ms_o)

def rot_sigma_e(i,j,phi,theta):
    sigma_e = sparse.identity(basis.Ns,dtype = complex)
    for k in range(1,int(L/2)):
        sigma_e = rot_sigma(2*k,i,j,phi,theta)@sigma_e
    return(sigma_e)

def rot_sigma_o(i,j,phi,theta):
    sigma_o = sparse.identity(basis.Ns,dtype = complex)
    for k in range(int(L/2)):
        sigma_o = rot_sigma(2*k+1,i,j,phi,theta)@sigma_o
    return(sigma_o)

def rot_sigma_all(i,j,phi,theta):
    sigma_all = sparse.identity(basis.Ns,dtype = complex)
    for k in range(L):
        sigma_all = rot_sigma(k,i,j,phi,theta)@sigma_all
    return(sigma_all)

In [46]:
def ms_hamiltonian(i,j,phi):
    sigma = np.zeros((2*l+1,2*l+1),dtype = complex)
    sigma[2*l-i][2*l-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2*l-j][2*l-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))

    sigma = sparse.csc_matrix(sigma)
    summe = np.zeros((basis.Ns,basis.Ns),dtype = complex)
    for k in range(L):
        if k == 0:
            tensprod = sigma
            for m in range(L-1):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
            tensprod = tensprod.toarray()
        else:
            tensprod = sparse.identity(2*l+1)
            for i in range(1,k):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
            tensprod = scipy.sparse.kron(tensprod,sigma)
            for i in range(k+1,L):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
            tensprod = tensprod.toarray()
        summe +=tensprod
    return summe@summe
                
def sigma_z_part(i,j):
    sigma = np.zeros((2*l+1,2*l+1),dtype = complex)
    sigma[2*l-j][2*l-j] = -1.
    sigma = sparse.csc_matrix(sigma)
    summe = 0
    for k in range(L):
        tensprod = 0
        if k==0:
            tensprod = sigma
            for i in range(L-1):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        else:
            tensprod = sparse.identity(2*l+1)
            for i in range(1,k):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
            tensprod = scipy.sparse.kron(tensprod,sigma)
            for i in range(k+1,L):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        summe +=tensprod
    return summe
    
matrix_part = ms_hamiltonian(1,2,0)
ms_ham_eigenval, ms_ham_eigenvec = scipy.linalg.eigh(matrix_part)




def ms_gate_global(theta):
    exponential = np.zeros((basis.Ns,basis.Ns),dtype = complex)
    for i in range(basis.Ns):
        exponential[i][i] = np.exp(1j*ms_ham_eigenval[i]*theta/4)
    exponential = scipy.sparse.csc_matrix(exponential)
    return ms_ham_eigenvec@exponential@ms_ham_eigenvec.T.conj()

    

In [47]:
#import timeit
#print(timeit.timeit('[ms_gate_global(0,2,np.pi)]', globals=globals()))
import datetime
print(datetime.datetime.now())
a = ms_gate_global(np.pi)
print(datetime.datetime.now())
print(np.size(a))


2022-03-28 12:19:35.772371
2022-03-28 12:19:35.780691
59049


In [48]:
opt_params = []
function_values = []
def callback_function(x,fun,context):
    opt_params.append(x)
    function_values.append(fun)


In [49]:
full_ham_matrix = sparse.csc_matrix(full_ham_matrix)

In [50]:
import optuna
Energy = np.zeros(6)
Fidelity = np.zeros(6)

#for N in range(6):
theta = np.zeros(5*N)
#cost function definition
def cost_function_sigma(trial):
    psi_var = psi_0
    for i in range(N):
        theta[5*i+0] = trial.suggest_float("5*"+str(i)+"+0", -5*np.pi, 5*np.pi)
        theta[5*i+1] = trial.suggest_float("5*"+str(i)+"+1", -2*np.pi, 2*np.pi)
        theta[5*i+2] = trial.suggest_float("5*"+str(i)+"+2", -2*np.pi, 2*np.pi)
        theta[5*i+3] = trial.suggest_float("5*"+str(i)+"+3", -2*np.pi, 2*np.pi)
        theta[5*i+4] = trial.suggest_float("5*"+str(i)+"+4", -2*np.pi, 2*np.pi)


        psi_var = ms_gate_global(theta[5*i+0])@psi_var
        psi_var = rot_sigma_e(0,2,0,theta[5*i+2])@rot_sigma_e(0,1,0,theta[5*i+1])@psi_var
        psi_var = rot_sigma_o(0,2,0,theta[5*i+4])@rot_sigma_o(0,1,0,theta[5*i+3])@psi_var
        psi_var = rot_sigma(0,0,2,0,theta[5*i+2])@rot_sigma(0,0,1,0,theta[5*i+1])@psi_var
        psi_var = rot_sigma(L-1,0,2,0,theta[5*i+2])@rot_sigma(L-1,0,1,0,theta[5*i+1])@psi_var

    cost = np.conj(psi_var)@constrained_full_ham_matrix@psi_var
    return np.real(cost)

if __name__ == "__main__":
    sampler = optuna.samplers.CmaEsSampler()
    study = optuna.create_study(sampler=sampler)
    study.optimize(cost_function_sigma, n_trials=3000)


theta = np.zeros(5*N)
i=0
for value in study.best_trial.params.items():
    theta[i] = value[1]
    i+=1

psi_var = psi_0
for i in range(N):

    psi_var = ms_gate_global(theta[5*i+0])@psi_var
    psi_var = rot_sigma_e(0,2,0,theta[5*i+2])@rot_sigma_e(0,1,0,theta[5*i+1])@psi_var
    psi_var = rot_sigma_o(0,2,0,theta[5*i+4])@rot_sigma_o(0,1,0,theta[5*i+3])@psi_var
    psi_var = rot_sigma(0,0,2,0,theta[5*i+2])@rot_sigma(0,0,1,0,theta[5*i+1])@psi_var
    psi_var = rot_sigma(L-1,0,2,0,theta[5*i+2])@rot_sigma(L-1,0,1,0,theta[5*i+1])@psi_var

Fidelity[N] = np.abs(np.dot(np.conj(psi_var),eigenvec[:,0]))**2
Energy[N] = np.conj(psi_var)@full_ham_matrix@psi_var

[I 2022-03-28 12:19:37,212] A new study created in memory with name: no-name-5e3568b8-83d6-4c9c-93ae-2d105bdc749f
[I 2022-03-28 12:19:37,271] Trial 0 finished with value: 5.126588568182271 and parameters: {'5*0+0': -3.598048698974269, '5*0+1': 3.745297007597097, '5*0+2': 5.8270846388994375, '5*0+3': 5.1756680811147096, '5*0+4': 5.753829167908341}. Best is trial 0 with value: 5.126588568182271.
[I 2022-03-28 12:19:37,320] Trial 1 finished with value: 1.007818829001632 and parameters: {'5*0+0': -0.5059923995088257, '5*0+1': -2.6179927185131455, '5*0+2': -1.304332721632912, '5*0+3': -0.40497887608117084, '5*0+4': -0.563832046501876}. Best is trial 1 with value: 1.007818829001632.
[I 2022-03-28 12:19:37,368] Trial 2 finished with value: 5.156145697546951 and parameters: {'5*0+0': -3.3884990094859737, '5*0+1': -0.31686728753670035, '5*0+2': -3.785820145043381, '5*0+3': 4.218825532489051, '5*0+4': -0.41908407172567713}. Best is trial 1 with value: 1.007818829001632.
[I 2022-03-28 12:19:37,41

[I 2022-03-28 12:19:38,446] Trial 27 finished with value: 0.050781204750377174 and parameters: {'5*0+0': -0.22662675171014557, '5*0+1': -1.2308939127441691, '5*0+2': -3.5864927880314736, '5*0+3': 1.7943381854903275, '5*0+4': -0.9846858051150809}. Best is trial 24 with value: -0.1614105083149647.
[I 2022-03-28 12:19:38,487] Trial 28 finished with value: 1.0901188330503184 and parameters: {'5*0+0': -0.23908285263202178, '5*0+1': 0.46515021472321416, '5*0+2': 0.32519750317018614, '5*0+3': 3.9935999644352034, '5*0+4': 1.0148512997922685}. Best is trial 24 with value: -0.1614105083149647.
[I 2022-03-28 12:19:38,528] Trial 29 finished with value: 2.8199823318151123 and parameters: {'5*0+0': 2.2676636210458594, '5*0+1': -2.017195144263206, '5*0+2': 0.6712408130557446, '5*0+3': 3.336929282648177, '5*0+4': 1.877693067212922}. Best is trial 24 with value: -0.1614105083149647.
[I 2022-03-28 12:19:38,580] Trial 30 finished with value: 0.9995345365257745 and parameters: {'5*0+0': 1.5701136484538547

[I 2022-03-28 12:19:39,799] Trial 54 finished with value: 0.7870396330796029 and parameters: {'5*0+0': 0.11609529526312634, '5*0+1': -2.8052185099601465, '5*0+2': -1.85073933310802, '5*0+3': -0.21693453350286251, '5*0+4': 0.8284504585427649}. Best is trial 35 with value: -0.30610749032498563.
[I 2022-03-28 12:19:39,849] Trial 55 finished with value: -0.03219577880035959 and parameters: {'5*0+0': -0.22238704008186927, '5*0+1': -0.5772022324995187, '5*0+2': -3.5360230294086157, '5*0+3': 1.3191771878907557, '5*0+4': -1.845197664187293}. Best is trial 35 with value: -0.30610749032498563.
[I 2022-03-28 12:19:39,900] Trial 56 finished with value: 0.6528763994612261 and parameters: {'5*0+0': -0.9310926519812932, '5*0+1': -0.7787904906463516, '5*0+2': -2.683757420267698, '5*0+3': 2.1513746429983818, '5*0+4': -1.9452624126254778}. Best is trial 35 with value: -0.30610749032498563.
[I 2022-03-28 12:19:39,951] Trial 57 finished with value: 0.2303359145548825 and parameters: {'5*0+0': 0.0069342349

[I 2022-03-28 12:19:41,101] Trial 81 finished with value: -0.2975521873205494 and parameters: {'5*0+0': 0.31183266196936144, '5*0+1': -1.1287087202772772, '5*0+2': -5.633708397804848, '5*0+3': -1.2812615984953128, '5*0+4': 1.8672809762315057}. Best is trial 58 with value: -0.4225386270103256.
[I 2022-03-28 12:19:41,144] Trial 82 finished with value: -0.09914968822799108 and parameters: {'5*0+0': 0.8932694704725976, '5*0+1': -1.0013339566250812, '5*0+2': -5.277416741595774, '5*0+3': 0.15966041504650735, '5*0+4': 2.2554548017887917}. Best is trial 58 with value: -0.4225386270103256.
[I 2022-03-28 12:19:41,187] Trial 83 finished with value: 0.5285997818613102 and parameters: {'5*0+0': 0.844695398807489, '5*0+1': -1.3614219918201185, '5*0+2': -3.458522952655554, '5*0+3': -1.9729487735588116, '5*0+4': -1.8226225867118593}. Best is trial 58 with value: -0.4225386270103256.
[I 2022-03-28 12:19:41,228] Trial 84 finished with value: -0.7197056126093427 and parameters: {'5*0+0': 0.02472942315497

[I 2022-03-28 12:19:42,362] Trial 108 finished with value: -0.5784864419800024 and parameters: {'5*0+0': -0.4341494126015625, '5*0+1': -1.7898436379946268, '5*0+2': -6.072986096723795, '5*0+3': -0.5637836682533428, '5*0+4': 0.16210289866857458}. Best is trial 84 with value: -0.7197056126093427.
[I 2022-03-28 12:19:42,405] Trial 109 finished with value: -0.6019723402652604 and parameters: {'5*0+0': -0.21405092052571373, '5*0+1': -1.4955840635974704, '5*0+2': -4.793199865013324, '5*0+3': -0.22114408187358212, '5*0+4': -1.1844308250708102}. Best is trial 84 with value: -0.7197056126093427.
[I 2022-03-28 12:19:42,444] Trial 110 finished with value: -0.39457491991553095 and parameters: {'5*0+0': 0.023954698770602578, '5*0+1': -0.6675830904493876, '5*0+2': -4.830997523135809, '5*0+3': 0.07221867921314085, '5*0+4': -1.3459876352858786}. Best is trial 84 with value: -0.7197056126093427.
[I 2022-03-28 12:19:42,491] Trial 111 finished with value: -0.22807806774397565 and parameters: {'5*0+0': -0

...

[I 2022-03-28 12:22:15,212] Trial 2889 finished with value: -0.7815399425345605 and parameters: {'5*0+0': 5.5677239752341665e-09, '5*0+1': -1.9247809004498158, '5*0+2': -6.283185233601806, '5*0+3': -0.04894374747170806, '5*0+4': 6.283180839016895}. Best is trial 2316 with value: -0.7815399425345607.
[I 2022-03-28 12:22:15,275] Trial 2890 finished with value: -0.7815399425345598 and parameters: {'5*0+0': 4.785344281535673e-09, '5*0+1': -1.9247809001765215, '5*0+2': -6.2831852479242425, '5*0+3': -0.048943744782601016, '5*0+4': 6.283180200192907}. Best is trial 2316 with value: -0.7815399425345607.
[I 2022-03-28 12:22:15,340] Trial 2891 finished with value: -0.7815399425345603 and parameters: {'5*0+0': 5.014975037940355e-09, '5*0+1': -1.924780899924281, '5*0+2': -6.283185246929242, '5*0+3': -0.048943744773044424, '5*0+4': 6.283180245759898}. Best is trial 2316 with value: -0.7815399425345607.
[I 2022-03-28 12:22:15,411] Trial 2892 finished with value: -0.7815399425345605 and parameters: {

[I 2022-03-28 12:22:17,035] Trial 2916 finished with value: -0.7815399425345602 and parameters: {'5*0+0': 4.625512697127931e-09, '5*0+1': -1.92478090015152, '5*0+2': -6.283185251817975, '5*0+3': -0.04894374397012283, '5*0+4': 6.28318002422573}. Best is trial 2316 with value: -0.7815399425345607.
[I 2022-03-28 12:22:17,097] Trial 2917 finished with value: -0.7815399425345603 and parameters: {'5*0+0': 5.3508466046437474e-09, '5*0+1': -1.9247809003950846, '5*0+2': -6.283185236357365, '5*0+3': -0.048943747011425266, '5*0+4': 6.28318072370751}. Best is trial 2316 with value: -0.7815399425345607.
[I 2022-03-28 12:22:17,158] Trial 2918 finished with value: -0.7815399425345598 and parameters: {'5*0+0': 5.341339356950558e-09, '5*0+1': -1.9247809005207785, '5*0+2': -6.283185236062146, '5*0+3': -0.04894374702669978, '5*0+4': 6.283180720850643}. Best is trial 2316 with value: -0.7815399425345607.
[I 2022-03-28 12:22:17,222] Trial 2919 finished with value: -0.7815399425345596 and parameters: {'5*0+

[I 2022-03-28 12:22:18,848] Trial 2943 finished with value: -0.7815399425345596 and parameters: {'5*0+0': 5.965859754576393e-09, '5*0+1': -1.9247809006369827, '5*0+2': -6.2831852230740965, '5*0+3': -0.048943749526673706, '5*0+4': 6.283181313498306}. Best is trial 2316 with value: -0.7815399425345607.
[I 2022-03-28 12:22:18,930] Trial 2944 finished with value: -0.7815399425345598 and parameters: {'5*0+0': 5.451198546805823e-09, '5*0+1': -1.9247809002012313, '5*0+2': -6.283185236969357, '5*0+3': -0.048943746624804385, '5*0+4': 6.28318068719606}. Best is trial 2316 with value: -0.7815399425345607.
[I 2022-03-28 12:22:19,012] Trial 2945 finished with value: -0.7815399425345603 and parameters: {'5*0+0': 5.5347598077604405e-09, '5*0+1': -1.9247809003440182, '5*0+2': -6.283185234115234, '5*0+3': -0.048943747289319696, '5*0+4': 6.283180814845596}. Best is trial 2316 with value: -0.7815399425345607.
[I 2022-03-28 12:22:19,094] Trial 2946 finished with value: -0.78153994253456 and parameters: {'

[I 2022-03-28 12:22:20,948] Trial 2970 finished with value: -0.7815399425345607 and parameters: {'5*0+0': 5.048950940149004e-09, '5*0+1': -1.9247809002469067, '5*0+2': -6.283185244731858, '5*0+3': -0.04894374522743542, '5*0+4': 6.2831803379580995}. Best is trial 2316 with value: -0.7815399425345607.
[I 2022-03-28 12:22:21,009] Trial 2971 finished with value: -0.7815399425345594 and parameters: {'5*0+0': 4.969953296175649e-09, '5*0+1': -1.924780900164401, '5*0+2': -6.283185247189775, '5*0+3': -0.048943744674014245, '5*0+4': 6.283180225164709}. Best is trial 2316 with value: -0.7815399425345607.
[I 2022-03-28 12:22:21,071] Trial 2972 finished with value: -0.78153994253456 and parameters: {'5*0+0': 5.249453081957962e-09, '5*0+1': -1.9247809003534713, '5*0+2': -6.283185240874317, '5*0+3': -0.048943746027379664, '5*0+4': 6.28318051209519}. Best is trial 2316 with value: -0.7815399425345607.
[I 2022-03-28 12:22:21,153] Trial 2973 finished with value: -0.7815399425345607 and parameters: {'5*0

[I 2022-03-28 12:22:22,983] Trial 2997 finished with value: -0.7815399425345596 and parameters: {'5*0+0': 5.52175009937958e-09, '5*0+1': -1.9247809007157726, '5*0+2': -6.283185232845064, '5*0+3': -0.04894374756768395, '5*0+4': 6.283180861130044}. Best is trial 2316 with value: -0.7815399425345607.
[I 2022-03-28 12:22:23,067] Trial 2998 finished with value: -0.7815399425345599 and parameters: {'5*0+0': 5.186660905736681e-09, '5*0+1': -1.924780900197179, '5*0+2': -6.283185243959618, '5*0+3': -0.04894374516595871, '5*0+4': 6.283180364308816}. Best is trial 2316 with value: -0.7815399425345607.
[I 2022-03-28 12:22:23,150] Trial 2999 finished with value: -0.7815399425345598 and parameters: {'5*0+0': 5.367941894817776e-09, '5*0+1': -1.9247809003077716, '5*0+2': -6.283185240823676, '5*0+3': -0.04894374591373802, '5*0+4': 6.283180514301257}. Best is trial 2316 with value: -0.7815399425345607.
<ipython-input-50-3ba6da7391bb>:49: ComplexWarning: Casting complex values to real discards the imagin

In [102]:
string = ""
for i in range(basis.Ns):
    if np.abs(psi_var[i])>=1e-5:
        string+=str(psi_var[i])+basis.int_to_state(basis.Ns-i-1)+"\n"
        #for j in range(basis.Ns):
            #if (np.abs(np.abs(eigenvectors[:,0][i])-np.abs(eigenvectors[:,0][j]))<=1e-5) & (i<j):
                #print(basis.int_to_state(basis.Ns-i-1),basis.int_to_state(basis.Ns-j-1))
print(string)

(0.00016029187456570332+0.0002972408900531397j)|2 2 2 2 2>
(-0.0005139108693679902-0.0006398039090896845j)|2 2 2 2 1>
(2.182372082802216e-05+4.671783062735368e-06j)|2 2 2 2 0>
(-0.0007395677306413118-5.245175893931513e-06j)|2 2 2 1 2>
(0.0017260494351292975-4.7212537555731016e-05j)|2 2 2 1 1>
(-0.0003075014445360338-7.600217592717656e-06j)|2 2 2 1 0>
(0.0002520734073255272-0.0003390796746367945j)|2 2 2 0 2>
(-0.0005036416473816452+0.0004089573884291266j)|2 2 2 0 1>
(8.013849223608494e-05+3.3563720931594754e-05j)|2 2 2 0 0>
(-0.0004028461394088125-0.0007470269179666111j)|2 2 1 2 2>
(0.0012915627213503282+0.0016079575802144578j)|2 2 1 2 1>
(-5.484745690823939e-05-1.1741142687427867e-05j)|2 2 1 2 0>
(0.0018586843901255983+1.3182195697823352e-05j)|2 2 1 1 2>
(-0.0043379139039483+0.00011865472618336346j)|2 2 1 1 1>
(0.0007728137819160511+1.910089531462531e-05j)|2 2 1 1 0>
(-0.0006335118312366026+0.0008521763080575332j)|2 2 1 0 2>
(0.0012657540741991859-0.0010277932400322592j)|2 2 1 0 1>
(-0

In [103]:
string = ""
for i in range(basis.Ns):
    if np.abs(eigenvectors[:,0][i])>=1e-5:
        string+=str(eigenvectors[:,0][i])+basis.int_to_state(basis.Ns-i-1)+"\n"
        #for j in range(basis.Ns):
            #if (np.abs(np.abs(eigenvectors[:,0][i])-np.abs(eigenvectors[:,0][j]))<=1e-5) & (i<j):
                #print(basis.int_to_state(basis.Ns-i-1),basis.int_to_state(basis.Ns-j-1))
print(string)

(0.002628581472633962+0.0031670282230540745j)|2 2 2 2 2>
(0.00983510649277315+0.011849760094421995j)|2 2 2 1 2>
(-0.018385327942305303-0.0221514353234178j)|2 2 2 1 1>
(0.00983510649277316+0.01184976009442201j)|2 1 2 2 2>
(0.046683140905168743+0.056245859725751586j)|2 1 2 1 2>
(-0.09097540837928116-0.10961109211971336j)|2 1 2 1 1>
(-0.09169869625438377-0.11048254051790495j)|2 1 1 1 2>
(0.20055017754408042+0.2416314955549615j)|2 1 1 1 1>
(-0.07561911980382367-0.09110917394591492j)|2 1 1 0 1>
(-0.01838532794230535-0.022151435323417804j)|1 1 2 2 2>
(-0.09097540837928114-0.10961109211971334j)|1 1 2 1 2>
(0.17863288691798104+0.21522459939884558j)|1 1 2 1 1>
(0.20055017754408042+0.2416314955549618j)|1 1 1 1 2>
(-0.4402640778925395-0.5304491318982665j)|1 1 1 1 1>
(0.16871345664936505+0.20327324238580102j)|1 1 1 0 1>
(-0.0756191198038237-0.09110917394591496j)|1 0 1 1 2>
(0.16871345664936496+0.2032732423858009j)|1 0 1 1 1>
(-0.07178029174557521-0.08648398848733291j)|1 0 1 0 1>
(-0.01309093803569